In [1]:
import py21cmfish as p21fish
import numpy as np
import shutil

import matplotlib as mpl
import matplotlib.pyplot as plt

from py21cmfish.power_spectra import *
from py21cmfish.io import *
from scipy import stats
from IPython.display import clear_output

/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:58: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn(
/n/home07/yitians/.conda/envs/dm21cm/lib/python3.11/site-packages/py21cmfast/_cfg.py:42: UserWarning: Your configuration file is out of date. Updating...
  warnings.warn("Your configuration file is out of date. Updating...")


# Set up 21cmFISH with Standard Astrophysics

In [3]:
noise_dir = '21cmSense_fid_EOS21/'
data_dir = '/n/holyscratch01/iaifi_lab/yitians/dm21cm/prod_outputs/bkg/LightCones/'
print('Loading data from',data_dir)


astro_params_vary = ['DM', 'F_STAR10', 'F_STAR7_MINI', 'ALPHA_STAR', 'ALPHA_STAR_MINI', 't_STAR',
                     'F_ESC10', 'F_ESC7_MINI', 'ALPHA_ESC', 'L_X', 'L_X_MINI', 'NU_X_THRESH', 'A_LW'
                    ]
default_param_values = [0, -1.25, -2.5, 0.5, 0.0, 0.5, -1.35, -1.35, -0.3, 40.5, 40.5, 500, 2.0]

astro_params_vary = list(np.array(astro_params_vary))
default_param_values = default_param_values
astro_params_fid = dict()
for i in range(len(astro_params_vary)):
    astro_params_fid[astro_params_vary[i]] = default_param_values[i]
    
    
# Load each parameter into a dictionary
params_EoS = {}

for param in astro_params_vary[1:]:
    params_EoS[param] = p21fish.Parameter(HII_DIM=128, BOX_LEN=256, param=param,
                                          output_dir=data_dir,
                                          PS_err_dir=noise_dir,
                                          new=False)
clear_output()
print('Done')

Done


In [4]:
params_EoS

{'F_STAR10': <py21cmfish.params.Parameter at 0x7fa83bd7e890>,
 'F_STAR7_MINI': <py21cmfish.params.Parameter at 0x7fa83c1469d0>,
 'ALPHA_STAR': <py21cmfish.params.Parameter at 0x7fa83c127010>,
 'ALPHA_STAR_MINI': <py21cmfish.params.Parameter at 0x7fa80aa6f190>,
 't_STAR': <py21cmfish.params.Parameter at 0x7fa80aa6d950>,
 'F_ESC10': <py21cmfish.params.Parameter at 0x7fa80aac0e10>,
 'F_ESC7_MINI': <py21cmfish.params.Parameter at 0x7fa80aac1910>,
 'ALPHA_ESC': <py21cmfish.params.Parameter at 0x7fa80abcb410>,
 'L_X': <py21cmfish.params.Parameter at 0x7fa80ab9bb90>,
 'L_X_MINI': <py21cmfish.params.Parameter at 0x7fa83c127890>,
 'NU_X_THRESH': <py21cmfish.params.Parameter at 0x7fa86d332650>,
 'A_LW': <py21cmfish.params.Parameter at 0x7fa80abb0210>}

# Do the Analysis At Each Mass

In [5]:
# The DM masses we use [eV\]
masses = np.array([15, 16, 17, 18], dtype=float)

# Lifetime scales calibrated for the Fisher Information
log_lifetimes = np.array([28.327, 28.793, 29.023, 29.053, 28.919, 28.656, 28.300, 27.887, 27.452, 27.032, 26.662])
log_f_s = np.log10(np.array([1e-10, 3e-7, 1e-2, 1e0]))
print(log_f_s)

[-10.          -6.52287875  -2.           0.        ]


In [6]:
data_dir

'/n/holyscratch01/iaifi_lab/yitians/dm21cm/prod_outputs/bkg/LightCones/'

In [7]:
dm_dir = '/n/holyscratch01/iaifi_lab/yitians/dm21cm/prod_outputs/pbh'

In [8]:
os.listdir(dm_dir)

['Mass_0', 'Mass_3', 'Mass_2', 'Mass_1']

In [9]:
# Gotta put the fiducial in each mass directory
for i in range(len(masses)):
    print(i)

    shutil.copyfile(f'{data_dir}/LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_fid_r54321.h5',
                    f'{dm_dir}/Mass_'+str(i) + '/LightCones/LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_fid_r54321.h5')
    
clear_output()
print('Done')

Done


In [10]:
params_EoS

{'F_STAR10': <py21cmfish.params.Parameter at 0x7fa83bd7e890>,
 'F_STAR7_MINI': <py21cmfish.params.Parameter at 0x7fa83c1469d0>,
 'ALPHA_STAR': <py21cmfish.params.Parameter at 0x7fa83c127010>,
 'ALPHA_STAR_MINI': <py21cmfish.params.Parameter at 0x7fa80aa6f190>,
 't_STAR': <py21cmfish.params.Parameter at 0x7fa80aa6d950>,
 'F_ESC10': <py21cmfish.params.Parameter at 0x7fa80aac0e10>,
 'F_ESC7_MINI': <py21cmfish.params.Parameter at 0x7fa80aac1910>,
 'ALPHA_ESC': <py21cmfish.params.Parameter at 0x7fa80abcb410>,
 'L_X': <py21cmfish.params.Parameter at 0x7fa80ab9bb90>,
 'L_X_MINI': <py21cmfish.params.Parameter at 0x7fa83c127890>,
 'NU_X_THRESH': <py21cmfish.params.Parameter at 0x7fa86d332650>,
 'A_LW': <py21cmfish.params.Parameter at 0x7fa80abb0210>}

In [13]:
astro_params_vary

['DM',
 'F_STAR10',
 'F_STAR7_MINI',
 'ALPHA_STAR',
 'ALPHA_STAR_MINI',
 't_STAR',
 'F_ESC10',
 'F_ESC7_MINI',
 'ALPHA_ESC',
 'L_X',
 'L_X_MINI',
 'NU_X_THRESH',
 'A_LW']

In [12]:
astro_params_vary

['DM',
 'F_STAR10',
 'F_STAR7_MINI',
 'ALPHA_STAR',
 'ALPHA_STAR_MINI',
 't_STAR',
 'F_ESC10',
 'F_ESC7_MINI',
 'ALPHA_ESC',
 'L_X',
 'L_X_MINI',
 'NU_X_THRESH',
 'A_LW']

In [14]:
os.listdir(f'{dm_dir}/Mass_'+str(mass_index)+'/LightCones/')

['global_signal_dict_F_STAR10.npy',
 'PS_deriv_DM_CDM.png',
 'LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_DM_1_r54321.h5',
 'global_signal_dict_DM.npy',
 'PS_z_HERA.npy',
 'LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_fid_r54321.h5',
 'GS_deriv_CDM_DM.png',
 'redshifts.npy',
 'LightCone_z5.0_HIIDIM=128_BOXLEN=256_fisher_DM_2_r54321.h5',
 'params_dict_DM.npy',
 'power_spectrum_deriv_dict_DM.npy',
 'power_spectrum_dict_DM.npy',
 'global_signal_deriv_dict_DM.npy',
 'params_dict_F_STAR10.npy',
 'lc_redshifts.npy',
 'power_spectrum_fid_21cmsense.npy']

In [ ]:
lims = []

for mass_index in np.arange(len(masses)):
    par = log_f_s[mass_index]
    
    for param in astro_params_vary:
        params_EoS[param] = p21fish.Parameter(HII_DIM=128, BOX_LEN=256, param=param,
                                              output_dir=f'{dm_dir}/Mass_'+str(mass_index)+'/LightCones/',
                                              PS_err_dir=noise_dir, new = False)
        # break

    
    Fij_matrix_PS, Finv_PS= p21fish.make_fisher_matrix(params_EoS,
                                                   fisher_params=astro_params_vary,
                                                   hpeak=0.0, obs='PS',
                                                   k_min=0.1, k_max=1,
                                                   sigma_mod_frac=0.2,
                                                   add_sigma_poisson=True, )
    lims.append(1 / (stats.chi2.ppf(.9, df = 1)**.5*np.sqrt(Finv_PS[0, 0]) * 10**(-par)))

clear_output()
print('Done')

In [6]:
np.save('./Results.npy', np.vstack((masses, lims)))